In [1]:
import pandas as pd
import numpy as np

from keras import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import SGD, Adam

import tensorflow as tf

import optuna

C:\Users\yveem\miniconda3\envs\machine_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### **Getting training & validation data**

In [2]:
df_train = pd.read_csv('../databases/training.csv')
df_valid = pd.read_csv('../databases/validation.csv')

In [3]:
X_train, y_train = df_train.drop(['IND_BOM_1_1'], axis=1), df_train['IND_BOM_1_1']
X_val, y_val = df_valid.drop(['IND_BOM_1_1'], axis=1), df_valid['IND_BOM_1_1']

In [4]:
X_train = np.array(X_train)
X_val = np.array(X_val)

#### **Parameters selection**

In [5]:
params = {
    'hidden_layer_units': [32, 128, 256],
    'hidden_layers': {
        'low': 1,
        'high': 2
    },
    'alpha': {
        'low': 0.001,
        'high': 0.1
    },
    'max_iter': [100, 150, 300],
    'batch_size': [32, 64, 128],
    'activation': ['tanh', 'relu', 'sigmoid'],
    'optimizer': ['SGD', 'Adam'],
    'loss_function': ['binary_crossentropy', 'mse'],
    'dropout_rate': {
        'low': 0.1,
        'high': 0.5
    }
}

reduce_lr = ReduceLROnPlateau(factor=0.5, patience=5, verbose=1)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
    min_delta=0.001
)

def objective(trial):
    model = Sequential()
    
    model.add(
        tf.keras.Input(shape=X_train.shape[1:])
    )

    hidden_layers = trial.suggest_int('hidden_layers', params['hidden_layers']['low'], params['hidden_layers']['high'])
    
    for _ in range(hidden_layers):
        model.add(
            Dense(
                units=trial.suggest_categorical(
                    'hidden_layer_units',
                    params['hidden_layer_units']
                ),
                activation=trial.suggest_categorical(
                    'activation',
                    params['activation']
                )
            )
        )

    model.add(
        Dropout(
            trial.suggest_float(
                'dropout_rate',
                params['dropout_rate']['low'],
                params['dropout_rate']['high'],
            )
        )
    )
    
    model.add(
        Dense(
            units=1,
            activation='sigmoid'
        )
    )

    alpha = trial.suggest_float(
        'alpha',
        params['alpha']['low'],
        params['alpha']['high']
    )
    optimizer_name = trial.suggest_categorical('optimizer', params['optimizer'])

    if optimizer_name == 'SGD':
        optimizer = SGD(learning_rate=alpha)
    elif optimizer_name == 'Adam':
        optimizer = Adam(learning_rate=alpha)

    model.compile(
        optimizer=optimizer,
        loss=trial.suggest_categorical('loss_function', params['loss_function']),
        metrics=['binary_accuracy']
    )

    model.fit(
        X_train,
        y_train,
        batch_size=trial.suggest_categorical(
            'batch_size',
            params['batch_size']
        ),
        epochs=trial.suggest_categorical(
            'max_iter',
            params['max_iter']
        ),
        validation_data=(X_val, y_val),
        callbacks=[early_stopping]
    )

    loss, accuracy = model.evaluate(X_val, y_val)

    return loss

In [6]:
n_trials = 20

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=n_trials)

[I 2023-04-29 11:43:22,362] A new study created in memory with name: no-name-847fea20-bf04-4010-a628-21d61712ae18


Epoch 1/150
2808/2808 [==============================] - 5s 2ms/step - loss: 0.2419 - binary_accuracy: 0.5750 - val_loss: 0.2330 - val_binary_accuracy: 0.6041
Epoch 2/150
2808/2808 [==============================] - 6s 2ms/step - loss: 0.2331 - binary_accuracy: 0.6121 - val_loss: 0.2295 - val_binary_accuracy: 0.6117
Epoch 3/150
2808/2808 [==============================] - 6s 2ms/step - loss: 0.2307 - binary_accuracy: 0.6196 - val_loss: 0.2238 - val_binary_accuracy: 0.6270
Epoch 4/150
2808/2808 [==============================] - 5s 2ms/step - loss: 0.2296 - binary_accuracy: 0.6242 - val_loss: 0.2257 - val_binary_accuracy: 0.6202
Epoch 5/150
2808/2808 [==============================] - 7s 2ms/step - loss: 0.2287 - binary_accuracy: 0.6253 - val_loss: 0.2223 - val_binary_accuracy: 0.6286
Epoch 6/150
2808/2808 [==============================] - 6s 2ms/step - loss: 0.2280 - binary_accuracy: 0.6277 - val_loss: 0.2273 - val_binary_accuracy: 0.6150
Epoch 7/150
2808/2808 [=======================

[I 2023-04-29 11:45:15,166] Trial 0 finished with value: 0.21986554563045502 and parameters: {'hidden_layers': 2, 'hidden_layer_units': 32, 'activation': 'relu', 'dropout_rate': 0.43113765575570817, 'alpha': 0.03444872012912931, 'optimizer': 'SGD', 'loss_function': 'mse', 'batch_size': 64, 'max_iter': 150}. Best is trial 0 with value: 0.21986554563045502.


Epoch 1/100
5616/5616 [==============================] - 13s 2ms/step - loss: 0.2442 - binary_accuracy: 0.5626 - val_loss: 0.2380 - val_binary_accuracy: 0.5927
Epoch 2/100
5616/5616 [==============================] - 11s 2ms/step - loss: 0.2367 - binary_accuracy: 0.5969 - val_loss: 0.2300 - val_binary_accuracy: 0.6196
Epoch 3/100
5616/5616 [==============================] - 12s 2ms/step - loss: 0.2340 - binary_accuracy: 0.6071 - val_loss: 0.2283 - val_binary_accuracy: 0.6214
Epoch 4/100
5616/5616 [==============================] - 13s 2ms/step - loss: 0.2325 - binary_accuracy: 0.6116 - val_loss: 0.2256 - val_binary_accuracy: 0.6268
Epoch 5/100
5616/5616 [==============================] - 10s 2ms/step - loss: 0.2318 - binary_accuracy: 0.6141 - val_loss: 0.2267 - val_binary_accuracy: 0.6235
Epoch 6/100
5616/5616 [==============================] - 11s 2ms/step - loss: 0.2309 - binary_accuracy: 0.6176 - val_loss: 0.2295 - val_binary_accuracy: 0.6142
Epoch 7/100
5616/5616 [=================

[I 2023-04-29 11:54:35,342] Trial 1 finished with value: 0.22074295580387115 and parameters: {'hidden_layers': 1, 'hidden_layer_units': 32, 'activation': 'sigmoid', 'dropout_rate': 0.23872141021158866, 'alpha': 0.04887805248668813, 'optimizer': 'SGD', 'loss_function': 'mse', 'batch_size': 32, 'max_iter': 100}. Best is trial 0 with value: 0.21986554563045502.


...

[I 2023-04-29 12:56:55,127] Trial 17 finished with value: 0.21424543857574463 and parameters: {'hidden_layers': 2, 'hidden_layer_units': 256, 'activation': 'sigmoid', 'dropout_rate': 0.19462650156548966, 'alpha': 0.08718431503862642, 'optimizer': 'SGD', 'loss_function': 'mse', 'batch_size': 32, 'max_iter': 150}. Best is trial 9 with value: 0.2129877209663391.


Epoch 1/300
5616/5616 [==============================] - 27s 5ms/step - loss: 0.7312 - binary_accuracy: 0.5002 - val_loss: 0.6957 - val_binary_accuracy: 0.3446
Epoch 2/300
5616/5616 [==============================] - 27s 5ms/step - loss: 0.6946 - binary_accuracy: 0.4990 - val_loss: 0.7142 - val_binary_accuracy: 0.3446
Epoch 3/300
5616/5616 [==============================] - 26s 5ms/step - loss: 0.6947 - binary_accuracy: 0.4995 - val_loss: 0.6894 - val_binary_accuracy: 0.6555
Epoch 4/300
5616/5616 [==============================] - 25s 4ms/step - loss: 0.6946 - binary_accuracy: 0.4998 - val_loss: 0.7146 - val_binary_accuracy: 0.3446
Epoch 5/300
5616/5616 [==============================] - 26s 5ms/step - loss: 0.6946 - binary_accuracy: 0.5014 - val_loss: 0.6849 - val_binary_accuracy: 0.6555
Epoch 6/300
5616/5616 [==============================] - 31s 5ms/step - loss: 0.6947 - binary_accuracy: 0.4983 - val_loss: 0.6964 - val_binary_accuracy: 0.3446
Epoch 7/300
5616/5616 [=================

[I 2023-04-29 13:10:35,643] Trial 18 finished with value: 0.6648998856544495 and parameters: {'hidden_layers': 2, 'hidden_layer_units': 256, 'activation': 'relu', 'dropout_rate': 0.19568878823973251, 'alpha': 0.06647238049565501, 'optimizer': 'Adam', 'loss_function': 'binary_crossentropy', 'batch_size': 32, 'max_iter': 300}. Best is trial 9 with value: 0.2129877209663391.


Epoch 1/100
5616/5616 [==============================] - 25s 4ms/step - loss: 0.2326 - binary_accuracy: 0.6113 - val_loss: 0.2451 - val_binary_accuracy: 0.5748
Epoch 2/100
5616/5616 [==============================] - 25s 4ms/step - loss: 0.2287 - binary_accuracy: 0.6227 - val_loss: 0.2391 - val_binary_accuracy: 0.5927
Epoch 3/100
5616/5616 [==============================] - 26s 5ms/step - loss: 0.2276 - binary_accuracy: 0.6249 - val_loss: 0.2359 - val_binary_accuracy: 0.6001
Epoch 4/100
5616/5616 [==============================] - 26s 5ms/step - loss: 0.2267 - binary_accuracy: 0.6282 - val_loss: 0.2378 - val_binary_accuracy: 0.5944
Epoch 5/100
5616/5616 [==============================] - 27s 5ms/step - loss: 0.2259 - binary_accuracy: 0.6306 - val_loss: 0.2306 - val_binary_accuracy: 0.6123
Epoch 6/100
5616/5616 [==============================] - 26s 5ms/step - loss: 0.2251 - binary_accuracy: 0.6320 - val_loss: 0.2368 - val_binary_accuracy: 0.5960
Epoch 7/100
5616/5616 [=================

[I 2023-04-29 13:21:39,793] Trial 19 finished with value: 0.20740830898284912 and parameters: {'hidden_layers': 2, 'hidden_layer_units': 256, 'activation': 'tanh', 'dropout_rate': 0.28993682622002964, 'alpha': 0.08742105156554487, 'optimizer': 'SGD', 'loss_function': 'mse', 'batch_size': 32, 'max_iter': 100}. Best is trial 19 with value: 0.20740830898284912.


#### **Saving study**

In [7]:
import joblib

In [8]:
save_path = './optuna_studies/mlp_study.pkl'

joblib.dump(study, save_path)

['./optuna_studies/mlp_study.pkl']